In [169]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# default='warn'
pd.options.mode.chained_assignment = None  
pd.set_option('display.max_columns', None)

%matplotlib inline

In [2]:
# import data
df = pd.read_csv('data.csv',delimiter=',')
df.head()

,invoice#,date,country,product_id,shop,gender,size_us,size_europe,size_uk,unit_price,discount,year,month,sale_price
0,52389,01-01-2014,United Kingdom,2152,UK2,Male,11.0,44,10.5,159.0,0%,2014,1,159.0
1,52390,01-01-2014,United States,2230,US15,Male,11.5,44-45,11.0,199.0,20%,2014,1,159.2
2,52391,01-01-2014,Canada,2160,CAN7,Male,9.5,42-43,9.0,149.0,20%,2014,1,119.2
3,52392,01-01-2014,United States,2234,US6,Female,9.5,40,7.5,159.0,0%,2014,1,159.0
4,52393,01-01-2014,United Kingdom,2222,UK4,Female,9.0,39-40,7.0,159.0,0%,2014,1,159.0


In [5]:
# check duplicated values
df.duplicated().all()

False

In [7]:
# check null values 
df.isna().all()

invoice#       False
date           False
country        False
product_id     False
shop           False
gender         False
size_us        False
size_europe    False
size_uk        False
unit_price     False
discount       False
year           False
month          False
sale_price     False
dtype: bool

In [10]:
# check datatype
df.dtypes

invoice#         int64
date            object
country         object
product_id       int64
shop            object
gender          object
size_us        float64
size_europe     object
size_uk        float64
unit_price     float64
discount        object
year             int64
month            int64
sale_price     float64
dtype: object

In [11]:
# country names
df.country.unique()

array(['United Kingdom', 'United States', 'Canada', 'Germany'],
      dtype=object)

In [12]:
# year
df.year.unique()

array([2014, 2015, 2016], dtype=int64)

In [217]:
# create sale data for each country and year 

def procreation(data,cntry,yr):
    aa = data.size_us.unique().tolist()
    aa.sort()
    dat = {'size_us':aa}
    df_new = pd.DataFrame(dat)
    d_n = pd.DataFrame(columns=range(1,13))
    df_new = df_new.append(d_n, ignore_index = True) 
    for mon in range(1,13):
        dd = data[(data['year']==yr) & (data['country']==cntry) & 
                       (data['month']==mon)]['size_us'].value_counts().tolist()
        dd1 = data[(data['year']==yr) & (data['country']==cntry) 
                   & (data['month']==mon)]['size_us'].value_counts().keys().tolist()
        dd11 = pd.DataFrame({'size':dd1,'count':dd})
        for ind , si in enumerate(df_new.size_us):
            for i,j in enumerate(dd11['size']):
                if si == j:
                    df_new[mon][ind] = dd11['count'][i]
    df_new.fillna(0,inplace=True)
    df_new.columns = ['size_us']+[str(x)+'_'+ str(yr) for x in ['jan','feb','mar','apr','may','jun',
                                                                'jul','aug','sep','oct','nov','dec']]
    return df_new


def procreation_shop(data,cntry,yr,shop):
    aa = data.size_us.unique().tolist()
    aa.sort()
    dat = {'size_us':aa}
    df_new = pd.DataFrame(dat)
    d_n = pd.DataFrame(columns=range(1,13))
    df_new = df_new.append(d_n, ignore_index = True) 
    for mon in range(1,13):
        dd = data[(data['year']==yr) & (data['country']==cntry) & 
                       (data['month']==mon) & (data['shop']==shop)]['size_us'].value_counts().tolist()
        dd1 = data[(data['year']==yr) & (data['country']==cntry) 
                   & (data['month']==mon) & (data['shop']==shop)]['size_us'].value_counts().keys().tolist()
        dd11 = pd.DataFrame({'size':dd1,'count':dd})
        for ind , si in enumerate(df_new.size_us):
            for i,j in enumerate(dd11['size']):
                if si == j:
                    df_new[mon][ind] = dd11['count'][i]
    df_new.fillna(0,inplace=True)
    df_new.columns = ['size_us']+[str(x)+'_'+ str(yr) for x in ['jan','feb','mar','apr','may','jun',
                                                                'jul','aug','sep','oct','nov','dec']]
    return df_new


def sum_all_country(data,cntry):
    count = 0
    for i in [2014,1015,2016]:
        if count == 0:
            df = procreation(data,cntry,i)
            df_OK = df.copy()
            count += 1
        else:
            df = procreation(data,cntry,i)
            df_OK = pd.merge(df_OK,df,on='size_us')
            
    return df_OK

def sum_all_shop(data,cntry,shop):
    count = 0
    for i in [2014,1015,2016]:
        if count == 0:
            df = procreation_shop(data,cntry,i,shop)
            df_OK = df.copy()
            count += 1
        else:
            df = procreation_shop(data,cntry,i,shop)
            df_OK = pd.merge(df_OK,df,on='size_us')
            
    return df_OK

#### We'll now generate the US sales data for the following years : 2014 , 2015 , 2016

In [220]:
# US sale data
df_US = sum_all_country(df,'United States')

In [221]:
df_US

,size_us,jan_2014,feb_2014,mar_2014,apr_2014,may_2014,jun_2014,jul_2014,aug_2014,sep_2014,oct_2014,nov_2014,dec_2014,jan_1015,feb_1015,mar_1015,apr_1015,may_1015,jun_1015,jul_1015,aug_1015,sep_1015,oct_1015,nov_1015,dec_1015,jan_2016,feb_2016,mar_2016,apr_2016,may_2016,jun_2016,jul_2016,aug_2016,sep_2016,oct_2016,nov_2016,dec_2016
0,4.5,0,0,0,0,0,0,0,0,0,3,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,0,3,0,1,1,0,0,0
1,5.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,0,0
2,5.5,0,1,0,2,0,0,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,1,0,3,4,0,1,0,3,6
3,6.0,1,2,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,4,6,10,5,3,3,4,4,9,3,1
4,6.5,0,0,0,2,0,1,3,0,3,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,13,8,8,1,2,9,13,9,2,3,3,1
5,7.0,0,1,3,4,4,1,2,4,2,4,8,12,0,0,0,0,0,0,0,0,0,0,0,0,6,3,4,5,11,16,9,8,8,8,4,4
6,7.5,16,2,4,0,3,4,3,2,4,8,4,0,0,0,0,0,0,0,0,0,0,0,0,0,6,16,10,25,19,19,20,13,22,24,10,5
7,8.0,24,15,12,15,6,10,7,16,9,12,5,4,0,0,0,0,0,0,0,0,0,0,0,0,26,43,24,23,46,21,40,42,36,38,25,20
8,8.5,4,4,6,15,19,5,4,8,8,11,4,7,0,0,0,0,0,0,0,0,0,0,0,0,19,17,20,20,42,34,52,34,33,31,22,21
9,9.0,8,12,15,12,17,9,12,2,17,12,8,17,0,0,0,0,0,0,0,0,0,0,0,0,37,24,23,19,36,39,47,46,55,49,27,30


#### We'll now generate the UK sales data for the following years : 2014 , 2015 , 2016

In [222]:
# UK sale data
df_UK = sum_all_country(df,'United Kingdom')

In [223]:
df_UK

,size_us,jan_2014,feb_2014,mar_2014,apr_2014,may_2014,jun_2014,jul_2014,aug_2014,sep_2014,oct_2014,nov_2014,dec_2014,jan_1015,feb_1015,mar_1015,apr_1015,may_1015,jun_1015,jul_1015,aug_1015,sep_1015,oct_1015,nov_1015,dec_1015,jan_2016,feb_2016,mar_2016,apr_2016,may_2016,jun_2016,jul_2016,aug_2016,sep_2016,oct_2016,nov_2016,dec_2016
0,4.5,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,4,2,1,0,0,0
1,5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,1,0,0,5,0,0,0
2,5.5,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,2
3,6.0,2,0,0,0,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,4,0,0,0,0,0,0,0
4,6.5,1,0,0,0,0,0,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,4,0,0,0,2,1,1,0,0
5,7.0,1,0,0,1,0,0,0,1,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,3,7,3,4,4,1,2,2,2,2
6,7.5,3,2,2,1,4,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,5,6,6,6,2,1,3,0,11,1,6,5
7,8.0,2,2,5,1,0,6,4,4,8,5,2,7,0,0,0,0,0,0,0,0,0,0,0,0,5,4,13,11,13,10,11,10,9,10,10,8
8,8.5,3,6,1,6,8,0,5,0,3,0,0,6,0,0,0,0,0,0,0,0,0,0,0,0,9,6,16,6,5,3,16,17,7,10,12,6
9,9.0,4,3,8,5,8,3,4,11,6,5,4,3,0,0,0,0,0,0,0,0,0,0,0,0,17,1,13,23,16,27,15,13,19,7,7,3


#### We'll now generate the Canada sales data for the following years : 2014 , 2015 , 2016

In [224]:
# Canada sale data
df_CANADA = sum_all_country(df,'Canada')

In [225]:
df_CANADA

,size_us,jan_2014,feb_2014,mar_2014,apr_2014,may_2014,jun_2014,jul_2014,aug_2014,sep_2014,oct_2014,nov_2014,dec_2014,jan_1015,feb_1015,mar_1015,apr_1015,may_1015,jun_1015,jul_1015,aug_1015,sep_1015,oct_1015,nov_1015,dec_1015,jan_2016,feb_2016,mar_2016,apr_2016,may_2016,jun_2016,jul_2016,aug_2016,sep_2016,oct_2016,nov_2016,dec_2016
0,4.5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0
1,5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2,0,0,0,0,0
2,5.5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
3,6.0,0,0,0,0,0,1,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,2,2,1,0,2,2,1,2,1
4,6.5,0,0,0,5,0,2,1,3,3,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,3,2,5,4,5,1,2,0,3
5,7.0,1,1,0,0,0,0,0,0,0,3,5,1,0,0,0,0,0,0,0,0,0,0,0,0,4,6,4,2,2,4,1,5,15,8,8,4
6,7.5,2,1,0,1,2,1,1,1,2,0,3,4,0,0,0,0,0,0,0,0,0,0,0,0,21,13,7,13,14,7,5,8,9,11,5,9
7,8.0,8,6,13,10,10,9,11,9,4,5,5,0,0,0,0,0,0,0,0,0,0,0,0,0,6,8,8,9,5,5,11,9,8,6,9,6
8,8.5,3,2,2,8,14,15,12,14,12,14,11,2,0,0,0,0,0,0,0,0,0,0,0,0,9,14,12,12,8,10,4,19,17,13,7,13
9,9.0,10,9,5,15,6,8,8,7,5,9,2,12,0,0,0,0,0,0,0,0,0,0,0,0,19,27,18,28,25,27,26,22,19,18,8,14


#### We'll now generate the Germany sales data for the following years : 2014 , 2015 , 2016

In [226]:
# Germany sale data
df_GERMANY = sum_all_country(df,'Germany')

In [227]:
df_GERMANY

,size_us,jan_2014,feb_2014,mar_2014,apr_2014,may_2014,jun_2014,jul_2014,aug_2014,sep_2014,oct_2014,nov_2014,dec_2014,jan_1015,feb_1015,mar_1015,apr_1015,may_1015,jun_1015,jul_1015,aug_1015,sep_1015,oct_1015,nov_1015,dec_1015,jan_2016,feb_2016,mar_2016,apr_2016,may_2016,jun_2016,jul_2016,aug_2016,sep_2016,oct_2016,nov_2016,dec_2016
0,4.5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3,0,0,0,1,1,0
1,5.0,0,0,0,0,2,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,3,0,0,0,0
2,5.5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2,1,1
3,6.0,1,1,1,1,0,0,0,3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,6,3,1,2,1,0,2,0,6,1,1
4,6.5,0,0,0,0,2,0,2,1,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,6,4,3,3,2,4,4,1,7,2,8,4
5,7.0,4,7,9,10,11,8,7,10,11,10,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,4,5,8,3,6,3,4,7,4,2,7
6,7.5,3,2,1,1,0,0,2,0,2,6,1,0,0,0,0,0,0,0,0,0,0,0,0,0,8,5,5,5,10,22,17,14,27,26,9,8
7,8.0,2,0,3,0,4,2,7,7,3,8,7,12,0,0,0,0,0,0,0,0,0,0,0,0,28,19,12,30,10,11,20,20,16,28,16,18
8,8.5,5,5,6,3,3,5,1,2,2,7,5,4,0,0,0,0,0,0,0,0,0,0,0,0,27,25,29,22,27,19,38,22,20,25,24,19
9,9.0,6,8,10,7,18,15,6,12,15,8,8,14,0,0,0,0,0,0,0,0,0,0,0,0,23,8,20,31,23,27,33,27,38,16,18,20


In [201]:
df.shop.unique()

array(['UK2', 'US15', 'CAN7', 'US6', 'UK4', 'GER2', 'CAN5', 'US13', 'UK1',
       'US1', 'US11', 'US2', 'GER1', 'CAN6', 'US7', 'CAN3', 'UK5', 'CAN2',
       'US5', 'CAN1', 'US12', 'US9', 'US4', 'US14', 'US3', 'UK3', 'GER3',
       'US8', 'US10'], dtype=object)